### MovieLens Dataset

In [1]:
!aws s3 cp s3://sagemaker-iris-yianc/athena-result/Unsaved/2020/12/11/6d64a6a3-e55a-4b92-866d-5addc9a4fc40.csv user-item.csv
!aws s3 cp s3://sagemaker-iris-yianc/athena-result/Unsaved/2020/12/11/e943298e-2a66-4a4f-8865-6531610caa91.csv product.csv



download: s3://sagemaker-iris-yianc/athena-result/Unsaved/2020/12/11/6d64a6a3-e55a-4b92-866d-5addc9a4fc40.csv to ./user-item.csv
download: s3://sagemaker-iris-yianc/athena-result/Unsaved/2020/12/11/e943298e-2a66-4a4f-8865-6531610caa91.csv to ./product.csv


In [2]:
import pandas as pd 
user_item = './user-item.csv'
item_path = './product.csv'



ui_df = pd.read_csv(user_item, nrows=1000000)
ui_df.head()

,product_id,product_category,customer_id,star_rating
0,B002CHXIAS,Shoes,3084118,5
1,B00H6CWE18,Shoes,3084118,5
2,B007JSGR9Y,Automotive,3084118,4
3,B00EKDDY88,Wireless,3084118,5
4,B0078XGE3Y,Wireless,3084118,5


In [3]:
# item_df = pd.read_csv(item_path, encoding = "ISO-8859-1")
item_df = pd.read_csv(item_path)
item_df.head()

,product_id,product_category,product_title
0,B0059LGBI0,PC,Sony PlayStation (PS) Vita Screen Protector [M...
1,B000CO7WPK,Automotive,KYB 344055 Excel-G Gas Shock
2,B00OQ01W9I,Wireless,Ravpower Customer Care
3,B00KKEI8ZA,Automotive,KAWELL® 2 Pack 16W LED Work Light ATV 4X4 Off ...
4,B007KPOZK4,Mobile_Apps,STARDOM: THE A-LIST


### Prepare to insert into Elastic Search

In [4]:
!pip install requests 
!pip install Elasticsearch==7.7.0 
!pip install urllib3

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 99 kB 1.8 MB/s  eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [5]:
master_user="master_user"
master_user_password="master_user_password"
elastic_search_endpoint="elastic_search_endpoint"

In [6]:
import boto3
import json
from elasticsearch import Elasticsearch, RequestsHttpConnection
def connectES(esEndPoint):
    print ('Connecting to the ES Endpoint {0}'.format(esEndPoint))
    try:
        esClient = Elasticsearch(
        hosts=[{'host': esEndPoint, 'port': 443}],
        http_auth=(master_user, master_user_password),
        use_ssl=True,
        verify_certs=True,
        connection_class=RequestsHttpConnection)
        return esClient
    except Exception as E:
        print("Unable to connect to {0}".format(esEndPoint))
        print(E)
        exit(3)

In [7]:
esClient = connectES(elastic_search_endpoint)

Connecting to the ES Endpoint search-amazon-review-demo-6z6ub2pwv5lfhpgvqnyxjb2g6q.us-east-1.es.amazonaws.com


In [23]:
indexDoc = {
    "dataRecord":{
    "iid": {"type":"text"},
    "title": {"type":"text", "index":"analyzed"}
},
"settings" : {
 "number_of_shards": 1,
 "number_of_replicas": 0
 }
}

In [24]:
def createIndex(esClient):
    try:
        res = esClient.indices.exists('product')
        print("Index Exists ... {}".format(res))
        if res is False:
            esClient.indices.create('movies', body=indexDoc)
        return 1
    except Exception as E:
        print("Unable to Create Index {0}".format("movies"))
        print(E)
        exit(4)
createIndex(esClient)        

Index Exists ... False
Unable to Create Index movies
RequestError(400, 'parse_exception', 'unknown key [dataRecord] for create index')


In [8]:
def indexDocElement(esClient, response):
    try:
        retval = esClient.index(index='product', doc_type='product', body=response)
    except Exception as E:
        print("Doc not indexed")
        print("Error: ",E)
        exit(5)

In [9]:
content = {"iid": "1", "title": "White Balloon, The (1995)"}
indexDocElement(esClient,content)

In [ ]:
item_df = item_df.fillna("")
item_arr = item_df.to_dict(orient="records")
import json

for i, j in enumerate(item_arr):
#     print(j)
    indexDocElement(esClient,response=j)

In [18]:
import requests
r = requests.get('https://{}/movies/movie/_search?q=title:black&size=100'.format(elastic_search_endpoint), auth=(master_user, master_user_password))
rjson = r.json()
rjson 

{'took': 163,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 4, 'relation': 'eq'},
  'max_score': 5.9481115,
  'hits': [{'_index': 'movies',
    '_type': 'movie',
    '_id': 'JYmbQXUBGYXIlCpo7g8f',
    '_score': 5.9481115,
    '_source': {'iid': 389,
     'title': 'Black Beauty (1994)',
     'release_date': '01-Jan-1994',
     'video_release_date': '',
     'imdb url': 'http://us.imdb.com/Title?Black+Beauty+(1994/I)',
     'unknown': 0,
     'Action': 0,
     'Adventure': 1,
     'Animation': 0,
     'Childrens': 1,
     'Comedy': 0,
     'Crime': 0,
     'Documentary': 0,
     'Drama': 0,
     'Fantasy': 0,
     'Film-Noir': 0,
     'Horror': 0,
     'Musical': 0,
     'Mystery': 0,
     'Romance': 0,
     'Sci-Fi': 0,
     'Thriller': 0,
     'War': 0,
     'Western': 0,
     'year': 1994}},
   {'_index': 'movies',
    '_type': 'movie',
    '_id': 'VTWbQXUBE17jUbYW7c0b',
    '_score': 5.8836293,
    '_source': {

### Save user_item_df and item_df for later usage

In [19]:
user_item_df.to_pickle("user_item_df.p")
item_df.to_pickle("item_df.p")
